In [ ]:
import torch
from oct_library import OCTProcessing
model_path = '/home/jetson/Documents/oct_embedded/model.pth'
oc_file = '/home/jetson/Documents/oct_embedded/dataset/hopkins/hc01_spectralis_macula_v1_s1_R.vol'
model = torch.load(model_path, map_location='cpu')
#model = model.half()
# # 'fbgemm' for server, 'qnnpack' for mobile
backend = 'qnnpack'

torch.backends.quantized.engine = backend

model.qconfig = torch.quantization.get_default_qconfig(backend)

# # insert observers
torch.quantization.prepare(model, inplace=True)

# # convert to quantized version
torch.quantization.convert(model, inplace=True)

In [ ]:
oct_process = OCTProcessing(oct_file=oc_file, torchmodel=model) # 125, 36, 10, 68, 15

In [ ]:
oct_process.plot_slo_fovea()

In [ ]:
oct_process.plot_overlay_oct_segmentation()
oct_process.plot_segmentation_full()
oct_process.get_individual_layers_segmentation(layer=[False, True, True, True, True, True, True,False, True, True])
oct_process.plot_selected_layers()